In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
l1 = keras.layers.Dense(units=1, input_shape=[1])
model = keras.Sequential([l1])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0,-1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict(np.array([10.0])))
print(f"Weights: {l1.get_weights()}")


In [ ]:
# Save the model
export_dir = 'saved_model/1'
model.export(export_dir)

In [ ]:
# Convert the saved model into TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Create the new model as a physical file
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

In [ ]:
# Without using an embedded system, we can test our model in our computer

# Load TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)


In [ ]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)

# Convert our input into a tensor our tflite model can understand
interpreter.set_tensor(input_details[0]['index'], to_predict)

# Run the prediction
interpreter.invoke()

# Convert our tensor output into something we can understand
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)